In [133]:
import pandas as pd

In [135]:
# Load the CSV file (with fallback encoding)
df = pd.read_csv('sales_data_sample.csv', encoding='ISO-8859-1')

In [163]:
# Step 1: Convert ORDERDATE to datetime
df['ORDERDATE'] = pd.to_datetime(df['ORDERDATE'], errors='coerce')

In [139]:
# Step 2: Remove duplicate rows
df_cleaned = df.drop_duplicates()

In [141]:
# Step 3: Strip whitespace from string columns
str_cols = df_cleaned.select_dtypes(include='object').columns
df_cleaned[str_cols] = df_cleaned[str_cols].apply(lambda x: x.str.strip())

In [143]:
# Step 4: Standardize column names
df_cleaned.columns = df_cleaned.columns.str.strip().str.lower().str.replace(' ', '_')

In [145]:
# Step 5: Check missing values
missing_values = df_cleaned.isnull().sum()
print(missing_values[missing_values > 0])

addressline2    2521
state           1486
postalcode        76
territory       1074
dtype: int64


In [147]:
# Step 6: Handle missing values
df_cleaned = df_cleaned.drop(columns=['addressline2'])  # Drop addressline2

In [149]:
# Fill state and territory with their mode (most common value)
df_cleaned['state'] = df_cleaned['state'].fillna(df_cleaned['state'].mode()[0])
df_cleaned['territory'] = df_cleaned['territory'].fillna(df_cleaned['territory'].mode()[0])

# Fill postalcode using forward fill
df_cleaned['postalcode'] = df_cleaned['postalcode'].ffill()

In [151]:
from sklearn.preprocessing import LabelEncoder
# Step 7: Encode categorical variable 'dealsize'
le = LabelEncoder()
df_cleaned['dealsize_encoded'] = le.fit_transform(df_cleaned['dealsize'])

In [153]:
# Step 8: Remove outliers from 'sales' using IQR method
Q1 = df_cleaned['sales'].quantile(0.25)
Q3 = df_cleaned['sales'].quantile(0.75)
IQR = Q3 - Q1

In [155]:
df_cleaned = df_cleaned[
    ~((df_cleaned['sales'] < (Q1 - 1.5 * IQR)) | (df_cleaned['sales'] > (Q3 + 1.5 * IQR)))
]


In [157]:
#Original Uncleaned Dataset
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2823 entries, 0 to 2822
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ORDERNUMBER       2823 non-null   int64         
 1   QUANTITYORDERED   2823 non-null   int64         
 2   PRICEEACH         2823 non-null   float64       
 3   ORDERLINENUMBER   2823 non-null   int64         
 4   SALES             2823 non-null   float64       
 5   ORDERDATE         2823 non-null   datetime64[ns]
 6   STATUS            2823 non-null   object        
 7   QTR_ID            2823 non-null   int64         
 8   MONTH_ID          2823 non-null   int64         
 9   YEAR_ID           2823 non-null   int64         
 10  PRODUCTLINE       2823 non-null   object        
 11  MSRP              2823 non-null   int64         
 12  PRODUCTCODE       2823 non-null   object        
 13  CUSTOMERNAME      2823 non-null   object        
 14  PHONE             2823 n

In [159]:
# Final summary
print(df_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
Index: 2742 entries, 0 to 2822
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ordernumber       2742 non-null   int64         
 1   quantityordered   2742 non-null   int64         
 2   priceeach         2742 non-null   float64       
 3   orderlinenumber   2742 non-null   int64         
 4   sales             2742 non-null   float64       
 5   orderdate         2742 non-null   datetime64[ns]
 6   status            2742 non-null   object        
 7   qtr_id            2742 non-null   int64         
 8   month_id          2742 non-null   int64         
 9   year_id           2742 non-null   int64         
 10  productline       2742 non-null   object        
 11  msrp              2742 non-null   int64         
 12  productcode       2742 non-null   object        
 13  customername      2742 non-null   object        
 14  phone             2742 non-nu

In [161]:
#Export the cleaned dataset
df_cleaned.to_csv("sales_data_cleaned.csv", index=False)